# **DLT Pipeline**

In [0]:
import dlt
from pyspark.sql.functions import *

**Streaming Table**

In [0]:
# Expectations
# คล้ายๆกับการเขียน SQL
my_rules = {
    "rule1": "product_id IS NOT NULL",
    "rule2": "product_name IS NOT NULL"
}

# Expectations


In [0]:
@dlt.table()

@dlt.expect_all_or_drop(my_rules)
def DimProducts_stage():
  df = spark.readStream.table("databricks_cata.silver.products_silver")
  return df

**Streaming View**

In [0]:
@dlt.view()
def DimProducts_View():
    # ถึงแม้ Table DimProducts_stage จะยังไม่ถูกสร้างขึ้นจริง(Cell ด้านบน) แต่เราสามารถ refer ไปที่ตารางนั้นได้ด้วย Live.<table_name>
    df = dlt.read_stream.table("Live.DimProducts_stage")
    return df
     

การเตรียมข้อมูลใน Delta Live Tables (DLT) โดยการสร้าง **@dlt.table()** เพื่ออ่านข้อมูลแบบ streaming ก่อน แล้วค่อยสร้าง** @dlt.view() **มีเหตุผลหลักๆ ดังนี้:
> 
1. การจัดการข้อมูลแบบ Streaming: การใช้ **@dlt.table() **เพื่ออ่านข้อมูลแบบ streaming ช่วยให้คุณสามารถจัดการกับข้อมูลที่เข้ามาอย่างต่อเนื่องได้อย่างมีประสิทธิภาพ โดยข้อมูลจะถูกประมวลผลทีละ record ซึ่งเหมาะสำหรับการทำงานกับแหล่งข้อมูลที่มีการเพิ่มข้อมูลใหม่อย่างต่อเนื่อง เช่น Kafka, Event Hubs หรือไฟล์ที่ถูกเพิ่มเข้ามาใน cloud storage

2. การตรวจสอบคุณภาพข้อมูล: การใช้ **@dlt.expect_all_or_drop(my_rules)** ใน **@dlt.table() **ช่วยให้คุณสามารถกำหนดกฎการตรวจสอบคุณภาพข้อมูลได้ โดยข้อมูลที่ไม่ผ่านกฎจะถูกดรอปออกไป ทำให้ข้อมูลที่เข้าสู่ขั้นตอนถัดไปมีคุณภาพที่ดีขึ้น

3. การแยกขั้นตอนการประมวลผล: การสร้าง **@dlt.view()** หลังจาก **@dlt.table()** ช่วยให้คุณสามารถแยกขั้นตอนการประมวลผลออกเป็นหลายๆ ขั้นตอน ทำให้โค้ดมีความชัดเจนและง่ายต่อการบำรุงรักษา โดย **@dlt.view()** จะทำหน้าที่เป็นการแปลงข้อมูลหรือการทำงานเพิ่มเติมที่ไม่ต้องการให้ข้อมูลถูกเก็บในตารางจริง

4. การใช้ประโยชน์จากการประมวลผลแบบ Batch และ Streaming: การใช้ **@dlt.table()** และ **@dlt.view()** ร่วมกันช่วยให้คุณสามารถใช้ประโยชน์จากการประมวลผลทั้งแบบ batch และ streaming ได้ โดย **@dlt.table()** จะทำหน้าที่เป็นตารางที่เก็บข้อมูลแบบ streaming และ **@dlt.view() **จะทำหน้าที่เป็นการแปลงข้อมูลหรือการทำงานเพิ่มเติมที่สามารถใช้ได้ทั้งในแบบ batch และ streaming

**DimProducts**

In [0]:
dlt.create_streaming_table("DimProducts")

In [0]:
# คำสั่ง apply_changes จะต้องทำงานกับ Delta Table ที่เป็น Streaming (หรือ Live Table)
dlt.apply_changes(

    target = "DimProducts",
    source = "Live.DimProducts_View",
    keys = ["product_id"],  # Primary key ของตาราง Source
    sequence_by = "product_id", # ควรที่จะเป็นคอลัมน์ Date ใน source แต่ถ้าไม่มีก็ให้เรียงลำดับตาม PK ก็ได้
    # ignore_null_updates = False,    
    # apply_as_deletes = None,
    # apply_as_truncates = None,
    # column_list = None,
    # except_column_list = None,
    stored_as_scd_type = 2, # SCD type 1 หรือ 2
    # track_history_column_list = None,
    # track_history_except_column_list = None
)

**การใช้ create_streaming_table() เพื่อสร้างตารางเป้าหมายสำหรับ apply_changes() มีเหตุผลหลักๆ ดังนี้:**

1. การจัดการข้อมูลแบบ Streaming: apply_changes() ใช้สำหรับการประมวลผลข้อมูลที่มีการเปลี่ยนแปลงอย่างต่อเนื่อง (Change Data Capture - CDC) ซึ่งต้องการตารางเป้าหมายที่รองรับการประมวลผลแบบ streaming เพื่อให้สามารถจัดการกับข้อมูลที่เข้ามาใหม่ได้อย่างมีประสิทธิภาพ

2. การเก็บประวัติการเปลี่ยนแปลง: การทำ SCD Type 2 ต้องการการเก็บประวัติการเปลี่ยนแปลงของข้อมูล ซึ่ง create_streaming_table() ช่วยให้สามารถเก็บข้อมูลที่มีการเปลี่ยนแปลงได้อย่างถูกต้องและครบถ้วน


3. การรองรับฟังก์ชันเฉพาะของ DLT: ฟังก์ชัน apply_changes() และ apply_changes_from_snapshot() ต้องการตารางเป้าหมายที่สร้างด้วย create_streaming_table() เพื่อให้สามารถทำงานได้อย่างถูกต้องและมีประสิทธิภาพ
